# Import Library

In [1]:
import configparser
import pandas as pd
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadDecoder
from pymodbus.payload import BinaryPayloadBuilder
from pymodbus.client.sync import ModbusTcpClient
%config Completer.use_jedi = False

In [2]:
Variable_Config_csv_Path = "VaribaleConfig.csv"
df = pd.read_csv(Variable_Config_csv_Path)
Server_Config_File_Path = "ServerConfig.ini"

server_ID_List = []
server_IP_List = []

In [3]:
def get_server_data(Server_Config_File_Path):

    server_config_object = configparser.ConfigParser()
    server_config_object.read(Server_Config_File_Path)

    server_ID_List.clear()
    server_IP_List.clear()
    
    for sect in server_config_object.sections():
        server_ID_List.append(sect)
    
    for s in server_ID_List:
        serverinfo = server_config_object[s]
        IP_Address = serverinfo["IP"]
        Port_Address = serverinfo["port"]
        server_IP_List.append(IP_Address)
        
    print("Server ID's : " + str(server_ID_List))
    print("Server IP's : " + str(server_IP_List))

In [4]:
def value_print(IP,DEV_ADD,DEV_IP,DATA_TYPE,VALUE):
    print ("IP Address : " + str(IP))
    print("Modbus Address : " + str(DEV_ADD))
    print("Modbus Device IP : " + str(DEV_IP))
    print("Modbus Address Data Type : " + str(DATA_TYPE))
    print("Value : " + str(VALUE))
    print("--" * 30)

In [5]:
get_server_data(Server_Config_File_Path)

Server ID's : ['SERVERCONFIG_1', 'SERVERCONFIG_2']
Server IP's : ['192.168.1.232', '127.0.0.1']


## Go to each section and read device address and read that address on modbus

In [6]:
for ip in server_IP_List: 
    client = ModbusTcpClient(ip)
    for i in range(0,len(df)):

        datatype = df["Data_Type"][i]
        IP_ADD = df["Com_Address"][i]
        if (datatype == "INT") and (IP_ADD == ip)  :
            value = client.read_holding_registers(int(df["Device_Address"][i]))
            decoder = BinaryPayloadDecoder.fromRegisters(value.registers,Endian.Big, wordorder=Endian.Little)
            value_print(ip,df["Device_Address"][i],df["Com_Address"][i],df["Data_Type"][i],decoder.decode_16bit_int())


        elif (datatype == "REAL") and (IP_ADD == ip) :
            value = client.read_holding_registers(int(df["Device_Address"][i]),2)
            decoder = BinaryPayloadDecoder.fromRegisters(value.registers,Endian.Big, wordorder=Endian.Little)
            value_print(ip,df["Device_Address"][i],df["Com_Address"][i],df["Data_Type"][i],decoder.decode_32bit_float())

        elif (datatype == "DINT") and (IP_ADD == ip) :
            value = client.read_holding_registers(int(df["Device_Address"][i]),2)
            decoder = BinaryPayloadDecoder.fromRegisters(value.registers,Endian.Big, wordorder=Endian.Little)
            value_print(ip,df["Device_Address"][i],df["Com_Address"][i],df["Data_Type"][i],decoder.decode_32bit_int())

        elif (datatype == "SWAP_REAL") and (IP_ADD == ip):
            value = client.read_holding_registers(int(df["Device_Address"][i]),2)
            decoder = BinaryPayloadDecoder.fromRegisters(value.registers,Endian.Big, wordorder=Endian.Big)
            value_print(ip,df["Device_Address"][i],df["Com_Address"][i],df["Data_Type"][i],decoder.decode_16bit_float())

        elif (datatype == "SWAP_DINT") and (IP_ADD == ip):
            value = client.read_holding_registers(int(df["Device_Address"][i]),2)
            decoder = BinaryPayloadDecoder.fromRegisters(value.registers,Endian.Big, wordorder=Endian.Big)
            value_print(ip,df["Device_Address"][i],df["Com_Address"][i],df["Data_Type"][i],decoder.decode_32bit_int())

IP Address : 192.168.1.232
Modbus Address : 12688
Modbus Device IP : 192.168.1.232
Modbus Address Data Type : REAL
Value : 34.560001373291016
------------------------------------------------------------
IP Address : 192.168.1.232
Modbus Address : 12690
Modbus Device IP : 192.168.1.232
Modbus Address Data Type : REAL
Value : 47.66999816894531
------------------------------------------------------------
IP Address : 192.168.1.232
Modbus Address : 12692
Modbus Device IP : 192.168.1.232
Modbus Address Data Type : REAL
Value : 0.0
------------------------------------------------------------
IP Address : 192.168.1.232
Modbus Address : 12694
Modbus Device IP : 192.168.1.232
Modbus Address Data Type : REAL
Value : 0.0
------------------------------------------------------------
IP Address : 192.168.1.232
Modbus Address : 12696
Modbus Device IP : 192.168.1.232
Modbus Address Data Type : REAL
Value : 0.0
------------------------------------------------------------
IP Address : 192.168.1.232
Modb

In [ ]:
client.close()